In [1]:
import pandas as pd
import numpy as np

In [2]:
DF_REALIZADO_2024 = pd.read_excel(
    "https://docs.google.com/spreadsheets/d/1rFuIVjNT6eo5wUr2nL7Y0uvM6r9eXgcaZPrA8Y91OsI/export?=download",
    sheet_name="Consolidado Central",
    usecols=[
        "Contrato",
        "Marca",
        "Unidade",
        "Valor",
        "NO",
        "Mês",
        "Nº FLUIG",
    ]
)

DF_NO_ATUALIZADO = pd.read_excel(
    "https://docs.google.com/spreadsheets/d/1FXJHfBeenCm7kDq-3SpJjuuBY0nXZ2xF-6DTK2HQyaQ/export?=download",
    sheet_name="NO",
    usecols=[
        "NO",
        "NO Ajuste",
        "Categoria",
        "Tipo Valor",
    ]
)

DF_UNIDADES_ATUALIZADAS = pd.read_excel(
    "https://docs.google.com/spreadsheets/d/1FXJHfBeenCm7kDq-3SpJjuuBY0nXZ2xF-6DTK2HQyaQ/export?=download",
    sheet_name="Unidades",
    usecols=[
        "Chave 2024",
        "Núcleo 2024",
        "Marca 2024",
        "Unidade 2024",
        "Regional",
    ]
)

In [48]:
DF_REALIZADO_2024_COPY = DF_REALIZADO_2024.copy()
DF_NO_ATUALIZADO_COPY = DF_NO_ATUALIZADO.copy()
DF_UNIDADES_ATUALIZADAS_COPY = DF_UNIDADES_ATUALIZADAS.copy()

DF_UNIDADES_ATUALIZADAS_COPY = DF_UNIDADES_ATUALIZADAS_COPY.drop_duplicates()
DF_NO_ATUALIZADO_COPY = DF_NO_ATUALIZADO_COPY.drop_duplicates()

DF_UNIDADES_ATUALIZADAS_COPY["Chave 2024"] = DF_UNIDADES_ATUALIZADAS_COPY["Chave 2024"].str.lower()

# Tratando base de realizado

In [49]:
# Separando dois DataFrames, um com erros (Marcas Vazias e Com valores), outro que já está padronizado, com Marcas e Valores
DF_ERROS_1 = DF_REALIZADO_2024_COPY.loc[(DF_REALIZADO_2024_COPY["Marca"].isna()) & (DF_REALIZADO_2024_COPY["Valor"] > 0)]
DF_REALIZADO_2024_COPY = DF_REALIZADO_2024_COPY.loc[(DF_REALIZADO_2024_COPY["Marca"].notna()) & (DF_REALIZADO_2024_COPY["Valor"] > 0)]

In [50]:
# Atualizando Marcas de Elite <UF> para Elite 
DF_REALIZADO_2024_COPY.loc[DF_REALIZADO_2024_COPY["Marca"].str.startswith("Elite"), "Marca"] = "Elite"

In [51]:
# Aplicando o De Para em Marcas, Unidades e Núcleo
DF_REALIZADO_2024_COPY["Chave 2024"] = DF_REALIZADO_2024_COPY["Marca"] + DF_REALIZADO_2024_COPY["Unidade"] 
DF_REALIZADO_2024_COPY["Chave 2024"] = DF_REALIZADO_2024_COPY["Chave 2024"].str.lower() 

DF_REALIZADO_2024_COPY = pd.merge(DF_REALIZADO_2024_COPY, DF_UNIDADES_ATUALIZADAS_COPY, how="left", on="Chave 2024")

In [52]:
DF_ERROS_2 = DF_REALIZADO_2024_COPY.loc[DF_REALIZADO_2024_COPY["Marca 2024"].isna()]
DF_REALIZADO_2024_COPY = DF_REALIZADO_2024_COPY.loc[DF_REALIZADO_2024_COPY["Marca 2024"].notna()]

In [53]:
DF_REALIZADO_2024_COPY = pd.merge(DF_REALIZADO_2024_COPY, DF_NO_ATUALIZADO_COPY, on="NO", how="left")

In [54]:
DF_ERROS_3 = DF_REALIZADO_2024_COPY[DF_REALIZADO_2024_COPY["NO Ajuste"].isna()]
DF_REALIZADO_2024_COPY = DF_REALIZADO_2024_COPY[DF_REALIZADO_2024_COPY["NO Ajuste"].notna()]
DF_REALIZADO_2024_COPY = DF_REALIZADO_2024_COPY[DF_REALIZADO_2024_COPY["NO Ajuste"] != "Desativado"]

In [55]:
DF_REALIZADO_2024_COPY["Tipo"] = "Real"
DF_REALIZADO_2024_COPY["Ano"] = 2024

DF_REALIZADO_2024_COPY = DF_REALIZADO_2024_COPY[[
    "Contrato",
    "Marca 2024",
    "Unidade 2024",
    "Mês",
    "Valor",
    "NO Ajuste",
    "Categoria",
    "Tipo Valor",
    "Núcleo 2024",
    "Tipo",
    "Regional",
    "Ano"
]]

In [56]:
DF_REALIZADO_2024_COPY["Valor"].sum()

77918650.91

In [43]:
DF_ERROS_1.to_excel("Valores sem Marca.xlsx")  # Preencher diretamente na planilha

In [58]:
DF_REALIZADO_2024_COPY

,Contrato,Marca 2024,Unidade 2024,Mês,Valor,NO Ajuste,Categoria,Tipo Valor,Núcleo 2024,Tipo,Regional,Ano
0,CS0171,Coleguium,Carlos Prates,1.0,41600.00,Obra de Reforma,Infraestrutura,CAPEX,Núcleo Coleguium,Real,1 - SP/MG/ES/PA,2024
1,CS0172,Coleguium,Gutierrez,1.0,14000.00,Obra de Reforma,Infraestrutura,CAPEX,Núcleo Coleguium,Real,1 - SP/MG/ES/PA,2024
2,CS0173,Coleguium,Gutierrez,1.0,35200.00,Obra de Reforma,Infraestrutura,CAPEX,Núcleo Coleguium,Real,1 - SP/MG/ES/PA,2024
3,CS0032,Coleguium,Alípio de Melo,1.0,70500.00,Obra de Reforma,Infraestrutura,CAPEX,Núcleo Coleguium,Real,1 - SP/MG/ES/PA,2024
4,CS0044,Coleguium,Internacional Belvedere,1.0,21600.00,Obra de Reforma,Infraestrutura,CAPEX,Núcleo Coleguium,Real,1 - SP/MG/ES/PA,2024
...,...,...,...,...,...,...,...,...,...,...,...,...
17183,CA1594,Pensi,Campos,6.0,1491.23,Alimentação dos Alunos - Integral,Supply,OPEX,Núcleo Pensi,Real,5 - RJ,2024
17184,CA1595,Pensi,Campos,6.0,118.12,Alimentação dos Alunos - Integral,Supply,OPEX,Núcleo Pensi,Real,5 - RJ,2024
17185,CA1596,Pensi,Campos,6.0,69.20,Alimentação dos Alunos - Integral,Supply,OPEX,Núcleo Pensi,Real,5 - RJ,2024
17186,CA1527,Pensi,Campos,6.0,2395.10,Alimentação dos Alunos - Integral,Supply,OPEX,Núcleo Pensi,Real,5 - RJ,2024


In [59]:
DF_ERROS_3["NO"].unique()

array(['Aluguel de Equipamentos', 'Multas Operacionais - Infração',
       'Consultoria não Recorrente', 'manut. predial corretiva',
       'Outras Despesas', 'Obra de Reforma|saldo 2023',
       'Água e Esgoto - ISA',
       'Obra de Reforma - Lista Manutenção Conservação Julho/24',
       'Apoio e Patrocínio', 'Outros - Mat. de Operação',
       'Obra de reforma'], dtype=object)

In [65]:
set([i for i in DF_ERROS_2["Chave 2024"].unique() if "definir" not in i])

{'45429tijuca 1',
 'alfacascavel',
 'alfasjp - ambiental',
 'coleguiumbh savassi',
 'coleguiumcidade nova',
 'elitebotafogo',
 'elitecentral',
 'elitemadureira',
 'eliteméier',
 'elitepato branco 2',
 'eliteshopping guadalupe',
 'elitetijuca 1',
 'holdingtijuca',
 'idealguará',
 'idealáguas claras',
 'núcleo coleguiumcarlos prates',
 'núcleo elitebarra',
 'núcleo elitetijuca 1',
 'pensibotafogo',
 'pensiicaraí',
 'pensiiguaçuano',
 'pensinilópolis',
 'pensirealengo',
 'pensirecreio',
 'pensisão joão de meriti',
 'pensitijuca',
 'pensivila valqueire',
 'phalpha campos dos goytacazes',
 'phitaipu',
 'phrecreio 1',
 'phtijuca',
 'phtrês rios'}

In [69]:
DF_REALIZADO_2024_COPY.to_excel("Realizado Atualizado.xlsx", index=False)